# Chapter 21: Traversable

## Traversable instances

In [1]:
:info Foldable

class Foldable (t :: * -> *) where
  Data.Foldable.fold :: Monoid m => t m -> m
  foldMap :: Monoid m => (a -> m) -> t a -> m
  foldr :: (a -> b -> b) -> b -> t a -> b
  Data.Foldable.foldr' :: (a -> b -> b) -> b -> t a -> b
  foldl :: (b -> a -> b) -> b -> t a -> b
  Data.Foldable.foldl' :: (b -> a -> b) -> b -> t a -> b
  foldr1 :: (a -> a -> a) -> t a -> a
  foldl1 :: (a -> a -> a) -> t a -> a
  Data.Foldable.toList :: t a -> [a]
  null :: t a -> Bool
  length :: t a -> Int
  elem :: Eq a => a -> t a -> Bool
  maximum :: Ord a => t a -> a
  minimum :: Ord a => t a -> a
  sum :: Num a => t a -> a
  product :: Num a => t a -> a
  {-# MINIMAL foldMap | foldr #-}
  	-- Defined in ‘Data.Foldable’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Foldable (Either a) -- Defined in ‘Data.Foldable’
instance Foldable ((,) a) -- Defined in ‘Data.Foldable’

In [2]:
:info Traversable

class (Functor t, Foldable t) => Traversable (t :: * -> *) where
  traverse :: Applicative f => (a -> f b) -> t a -> f (t b)
  sequenceA :: Applicative f => t (f a) -> f (t a)
  mapM :: Monad m => (a -> m b) -> t a -> m (t b)
  sequence :: Monad m => t (m a) -> m (t a)
  {-# MINIMAL traverse | sequenceA #-}
  	-- Defined in ‘Data.Traversable’
instance Traversable [] -- Defined in ‘Data.Traversable’
instance Traversable Maybe -- Defined in ‘Data.Traversable’
instance Traversable (Either a) -- Defined in ‘Data.Traversable’
instance Traversable ((,) a) -- Defined in ‘Data.Traversable’

In [3]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

instance Foldable Identity where
    foldr ab2b b0 (Identity a) = ab2b a b0

instance Traversable Identity where
    traverse f (Identity a) = Identity <$> f a

In [4]:
newtype Constant a b = Constant { getConstant :: a }

instance Functor (Constant a) where
    fmap _ (Constant a) = Constant a
    
instance Foldable (Constant a) where
    foldr _ c0 _ = c0
    
instance Traversable (Constant a) where
    traverse _ (Constant a) = pure (Constant a)

In [5]:
data Optional a
    = Nada
    | Yep a
    
instance Functor Optional where
    fmap _ Nada = Nada
    fmap a2b (Yep a) = Yep $ a2b a
    
instance Foldable Optional where
    foldr _ b0 Nada = b0
    foldr ab2b b0 (Yep a) = ab2b a b0
    
instance Traversable Optional where
    traverse _ Nada = pure Nada
    traverse f (Yep a) = Yep <$> f a

In [6]:
import Control.Applicative (liftA2)

data List a
    = Nil
    | Cons a (List a)
    
instance Functor List where
    fmap _ Nil = Nil
    fmap f (Cons a as) = Cons (f a) (fmap f as)
    
instance Foldable List where
    foldr _ b0 Nil = b0
    foldr ab2b b0 (Cons a as) = ab2b a (foldr ab2b b0 as)
    
instance Traversable List where
    traverse _ Nil = pure Nil
    traverse f (Cons a as) = liftA2 Cons fa fas where
        fa = f a
        fas = traverse f as

In [7]:
data Three a b c = Three a b c

instance Functor (Three a b) where
    fmap c2d (Three a b c) = Three a b (c2d c)
    
instance Foldable (Three a b) where
    foldr cd2d d0 (Three _ _ c) = cd2d c d0
    
instance Traversable (Three a b) where
    traverse f (Three a b c) = Three a b <$> f c

In [8]:
data Three' a b = Three' a b b

instance Functor (Three' a) where
    fmap b2c (Three' a b1 b2) = Three' a (b2c b1) (b2c b2)
    
instance Foldable (Three' a) where
    foldr bc2c c0 (Three' _ b1 b2) = bc2c b2 . bc2c b1 $ c0

instance Traversable (Three' a) where
    traverse f (Three' a b1 b2) = liftA2 (Three' a) (f b1) (f b2)

In [9]:
data S n a = S (n a) a

instance Functor n => Functor (S n) where
    fmap f (S na a) = S (f <$> na) (f a)
    
instance Foldable n => Foldable (S n) where
    foldr ab2b b0 (S na a) = foldr ab2b (ab2b a b0) na
    
instance Traversable n => Traversable (S n) where
    traverse f (S na a) = liftA2 S (traverse f na) (f a)

## Instances for Tree

In [10]:
import Control.Applicative (liftA3)

data Tree a
    = Empty
    | Leaf a
    | Node (Tree a) a (Tree a)
    deriving (Eq, Show)
    
instance Functor Tree where
    fmap _ Empty = Empty
    fmap a2b (Leaf a) = Leaf $ a2b a
    fmap a2b (Node t1 a t2) = Node (fmap a2b t1) (a2b a) (fmap a2b t2)
    
instance Foldable Tree where
    foldMap _ Empty = mempty
    foldMap a2m (Leaf a) = a2m a
    foldMap a2m (Node t1 a t2) = foldMap a2m t1 <> a2m a <> foldMap a2m t2
    
instance Traversable Tree where
    traverse _ Empty = pure Empty
    traverse f (Leaf a) = Leaf <$> f a
    traverse f (Node t1 a t2) = liftA3 Node (traverse f t1) (f a) (traverse f t2)